<a href="https://colab.research.google.com/github/DGuilherme/PMTese/blob/main/TransformerTSKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow scikit-learn

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [8]:
!pip install tensorflow pandas scikit-learn

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from google.colab import drive
drive.mount('/content/drive')

# Load data
column_names = ['id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor{i}' for i in range(1, 22)]
train_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/train_FD001.txt/train_FD001.txt', delim_whitespace=True, header=None)
test_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/test_FD001.txt/test_FD001.txt', delim_whitespace=True, header=None)
rul_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/RUL_FD001.txt/RUL_FD001.txt', delim_whitespace=True, header=None)

train_data.columns = column_names
test_data.columns = column_names
rul_data.columns = ['RUL']

# Normalize sensor data
scaler = StandardScaler()
train_data.iloc[:, 5:] = scaler.fit_transform(train_data.iloc[:, 5:])
test_data.iloc[:, 5:] = scaler.transform(test_data.iloc[:, 5:])

# Generate RUL for training data
max_cycle = train_data.groupby('id')['cycle'].max()
train_data = train_data.merge(max_cycle.reset_index(), on='id', suffixes=('', '_max'))
train_data['RUL'] = train_data['cycle_max'] - train_data['cycle']
train_data.drop(columns=['cycle_max'], inplace=True)

# RUL for test data
max_cycle_test = test_data.groupby('id')['cycle'].max().reset_index()
max_cycle_test.columns = ['id', 'max_cycle']
rul_data.columns = ['RUL']
max_cycle_test['RUL'] = rul_data['RUL']
test_data = test_data.merge(max_cycle_test, on='id')
test_data['RUL'] = test_data['RUL'] + test_data['max_cycle'] - test_data['cycle']
test_data.drop(columns=['max_cycle'], inplace=True)

useful_sensor_cols = ['setting1', 'setting2', 'setting3'] + \
    [f'sensor{i}' for i in [2, 3, 4, 7, 8, 11, 12, 13, 14, 15, 17, 20, 21]]

train_data = train_data[['id', 'cycle'] + useful_sensor_cols + ['RUL']]
test_data = test_data[['id', 'cycle'] + useful_sensor_cols + ['RUL']]

# Create sequences
def create_sequences(data, seq_length=50):
    sequences = []
    for unit in data['id'].unique():
        unit_data = data[data['id'] == unit].reset_index(drop=True)
        for start in range(len(unit_data) - seq_length + 1):
            end = start + seq_length
            seq_X = unit_data.iloc[start:end, 2:-1].values  # all sensor + setting cols
            seq_y = unit_data.iloc[end-1]['RUL']
            sequences.append((seq_X, seq_y))
    return sequences

seq_length = 50
train_seqs = create_sequences(train_data, seq_length)
test_seqs = create_sequences(test_data, seq_length)

train_X, val_X, train_y, val_y = train_test_split(
    [seq[0] for seq in train_seqs],
    [seq[1] for seq in train_seqs],
    test_size=0.2,
    random_state=42
)

# Convert to numpy arrays
train_X = np.array(train_X)
train_y = np.array(train_y)
val_X = np.array(val_X)
val_y = np.array(val_y)
test_X = np.array([seq[0] for seq in test_seqs])
test_y = np.array([seq[1] for seq in test_seqs])

# Define Transformer
class TimeSeriesTransformer(tf.keras.Model):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TimeSeriesTransformer, self).__init__()
        self.input_proj = tf.keras.layers.Dense(model_dim)

        self.encoder_layers = []
        for _ in range(num_layers):
            self.encoder_layers.append([
                tf.keras.layers.LayerNormalization(),
                tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=model_dim),
                tf.keras.Sequential([
                    tf.keras.layers.Dense(model_dim * 4, activation='relu'),
                    tf.keras.layers.Dense(model_dim)
                ]),
            ])

        self.global_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.output_layer = tf.keras.layers.Dense(output_dim)

    def call(self, inputs, training=False):
        x = self.input_proj(inputs)

        for norm, mha, ffn in self.encoder_layers:
            attn_output = mha(x, x)
            x = norm(x + attn_output)
            ff_output = ffn(x)
            x = norm(x + ff_output)

        x = self.global_pool(x)
        return self.output_layer(x)

# Instantiate model
input_dim = train_X.shape[2]
model_dim = 64
num_heads = 8
num_layers = 4
output_dim = 1

model = TimeSeriesTransformer(input_dim, model_dim, num_heads, num_layers, output_dim)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae'])

# Train
history = model.fit(
    train_X, train_y,
    validation_data=(val_X, val_y),
    epochs=20,
    batch_size=32
)

# Predict
predictions = model.predict(test_X)

# Evaluate


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-8-dc04ddf23ab2>:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/train_FD001.txt/train_FD001.txt', delim_whitespace=True, header=None)
<ipython-input-8-dc04ddf23ab2>:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  test_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/test_FD001.txt/test_FD001.txt', delim_whitespace=True, header=None)
<ipython-input-8-dc04ddf23ab2>:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  rul_data = pd.read_csv('/content/drive/MyDrive/Python/predictive-maintenance-main/datasets/cmapss/RUL_FD001.txt/RUL_FD001.txt'

Epoch 1/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 233s 548ms/step - loss: 7464.1372 - mae: 67.4842 - val_loss: 3732.0640 - val_mae: 47.2499
Epoch 2/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 251s 519ms/step - loss: 3556.4678 - mae: 46.9255 - val_loss: 3256.3074 - val_mae: 46.0064
Epoch 3/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 203s 516ms/step - loss: 3332.2788 - mae: 46.8124 - val_loss: 3255.3337 - val_mae: 45.9743
Epoch 4/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 261s 515ms/step - loss: 3282.3406 - mae: 46.2654 - val_loss: 3258.6279 - val_mae: 46.0703
Epoch 5/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 255s 498ms/step - loss: 3316.8347 - mae: 46.7112 - val_loss: 3257.1235 - val_mae: 46.0310
Epoch 6/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 209s 516ms/step - loss: 3339.1941 - mae: 46.8365 - val_loss: 3268.8042 - val_mae: 46.2598
Epoch 7/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 256s 502ms/step - loss: 3278.3350 - mae: 46.4767 - val_loss: 3260.7627 - val_mae: 46.1166
Epoch 8/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 206s 514ms/step - loss: 3307.0974 - mae: 46.5382 -

In [9]:
rmse = np.sqrt(mean_squared_error(test_y, predictions))
mae = mean_absolute_error(test_y, predictions)
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

# Optional: save model
model.save('/content/drive/MyDrive/transformer_rul_model_tf.keras')

Test RMSE: 60.4115
Test MAE: 47.9250
